In [ ]:
# Importing necessary libraries
import torch  # PyTorch core library for tensor operations
import torch.nn as nn  # Neural network functionalities from PyTorch
import pytorch_lightning as pl  # High-level wrapper for PyTorch to automate training and logging
from torch.utils.data import DataLoader  # DataLoader helps in batching data
from Data_loader_1DCXNet import UBFC_LOADER  # Custom data loader for UBFC dataset

import torch.nn.functional as F  # Functional API of PyTorch for various operations like activation functions
import scipy  # Scientific library for technical computing
from scipy import signal  # Signal processing functionalities from SciPy
# import heartpy as hp  # Heart rate extraction library (commented out in the current version)
from pytorch_lightning.callbacks import ModelCheckpoint  # Callback to save the best model during training
from pytorch_lightning.loggers import TensorBoardLogger  # Logger for visualizing metrics in TensorBoard
import numpy as np  # Numerical operations library for handling arrays and matrices
import matplotlib.pyplot as plt  # Plotting library for visualizing data and metrics


In [ ]:
# Function to find the heart rate (HR) from a given waveform using FFT (Fast Fourier Transform)
def HR_FINDER(wave, fs):
    # Perform FFT to get the frequency components of the signal, considering only the first 500 frequencies
    fftData = np.fft.fft(wave, 1000)[0:500]
    
    # Generate an array of corresponding frequencies (Hz) for the FFT result
    hz = np.linspace(0, fs / 2, int(len(fftData)))
    
    # Compute the power spectrum (square of the absolute FFT values)
    powerSpectrum = np.abs(fftData) ** 2
    
    # Find the frequency with the maximum power (dominant frequency)
    maxFreq = np.argmax(powerSpectrum)
    
    # Compute the heart rate in beats per minute (bpm)
    HR = hz[maxFreq] * 60
    return HR


# Function to filter the signal within the specified frequency range (bandpass filter)
def filtering(data, fs, lowerCutoff=1, higherCutoff=3, filterOrder=5):
    # Convert cutoff frequencies to digital frequency based on the Nyquist theorem
    lowerCutoffDigital = lowerCutoff / (0.5 * fs)
    higherCutoffDigital = higherCutoff / (0.5 * fs)
    
    # Design a bandpass Butterworth filter with the specified order and cutoff frequencies
    b, a = signal.butter(filterOrder, [lowerCutoffDigital, higherCutoffDigital], btype='band', analog=False)
    
    # Apply the filter to the input signal using the filtfilt function (zero-phase filtering)
    filtsignal = signal.filtfilt(b, a, data)
    
    return filtsignal


In [ ]:
train_path=r'Train_path' ## training path
val_path=r'val_path' ### validation path

In [ ]:
# Load training data from the specified train path
train_data = UBFC_LOADER(train_path)  

# Load validation data from the specified validation path
val_data = UBFC_LOADER(val_path)  


In [ ]:
# Create a DataLoader for the training data with a batch size of 10.
# The data is shuffled and the last incomplete batch is dropped.
train_loader = DataLoader(train_data, batch_size=10, shuffle=True, drop_last=True)

# Create a DataLoader for the validation data with a batch size of 10.
# The data is not shuffled and the last incomplete batch is dropped.
val_loader = DataLoader(val_data, batch_size=10, shuffle=False, drop_last=True)


# Model-1D-CXNet

In [ ]:
class Transformer_ppg(nn.Module):
    def __init__(self):
        super().__init__()

        # First convolutional block
        self.cnn1 = nn.Sequential(
            nn.Conv1d(in_channels=9, out_channels=64, kernel_size=3, padding=1),  # 1D convolution with 9 input channels and 64 output channels
            nn.BatchNorm1d(64),  # Batch normalization
            nn.ReLU(),  # ReLU activation function
            nn.MaxPool1d(kernel_size=1),  # Max pooling with kernel size of 1
        )

        # Second convolutional block
        self.cnn2 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1),  # 1D convolution with 64 input and output channels
            nn.BatchNorm1d(64),  # Batch normalization
            nn.ReLU(),  # ReLU activation function
            nn.MaxPool1d(kernel_size=1),  # Max pooling with kernel size of 1
        )

        # Third convolutional block
        self.cnn3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1),  # 1D convolution
            nn.BatchNorm1d(64),  # Batch normalization
            nn.ReLU(),  # ReLU activation function
            nn.MaxPool1d(kernel_size=1),  # Max pooling with kernel size of 1
        )

        # Global Average Pooling to reduce dimensions to a single value
        self.global_pool = nn.AdaptiveAvgPool1d(output_size=(1))

        # Max pooling layer
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)

        # First convolution block with additional convolutions and batch normalization
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        # First Transformer Encoder layer
        encoder_layer1 = nn.TransformerEncoderLayer(d_model=150, nhead=1, batch_first=True)
        self.transformer_model1 = nn.TransformerEncoder(encoder_layer1, num_layers=2)

        # Second convolution block with additional convolutions and batch normalization
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        # Second Transformer Encoder layer
        encoder_layer2 = nn.TransformerEncoderLayer(d_model=150, nhead=1, batch_first=True)
        self.transformer_model2 = nn.TransformerEncoder(encoder_layer2, num_layers=2)

        # Third convolution block with additional convolutions and batch normalization
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        # Third Transformer Encoder layer
        encoder_layer3 = nn.TransformerEncoderLayer(d_model=150, nhead=1, batch_first=True)
        self.transformer_model3 = nn.TransformerEncoder(encoder_layer3, num_layers=2)

        # Final convolution layer to output the predicted result
        self.end = nn.Conv1d(in_channels=64, out_channels=1, kernel_size=3, stride=1, padding=1)

        # Dropout layer to prevent overfitting
        self.drop_out = nn.Dropout(p=0.3)

        # ReLU activation function
        self.relu = nn.ReLU()

    def forward(self, face):
        # Pass input through the CNN layers
        forehead = self.cnn1(face)
        forehead = self.cnn2(forehead)
        forehead = self.cnn3(forehead)

        # Pass through first convolution + transformer block
        signal = self.conv1(forehead)
        signal = self.transformer_model1(signal)
        signal = self.maxpool(signal)
        signal = F.interpolate(signal, scale_factor=2)  # Upsample the signal
        signal = self.drop_out(signal)

        # Pass through second convolution + transformer block
        signal = self.conv2(signal)
        signal = self.transformer_model2(signal)
        signal = self.maxpool(signal)
        signal = F.interpolate(signal, scale_factor=2)  # Upsample the signal
        signal = self.drop_out(signal)

        # Pass through third convolution + transformer block
        signal = self.conv3(signal)
        signal = self.transformer_model3(signal)
        signal = self.maxpool(signal)
        signal = F.interpolate(signal, scale_factor=2)  # Upsample the signal
        signal = self.drop_out(signal)

        # Final output convolution layer
        signal = self.end(signal)

        # Return the output after squeezing the extra dimensions
        return torch.squeeze(signal)


In [ ]:
# Function to calculate the Pearson correlation coefficient between two signals
def Pearson_loss(signal1, signal2):
    # Calculate the mean of both signals
    mean_signal1 = torch.mean(signal1)
    mean_signal2 = torch.mean(signal2)
    
    # Calculate the numerator of the Pearson correlation (covariance between the signals)
    num = torch.sum((signal1 - mean_signal1) * (signal2 - mean_signal2))
    
    # Calculate the denominator of the Pearson correlation (standard deviation of both signals)
    dem1 = torch.sqrt(torch.sum((signal1 - mean_signal1)**2))  # Standard deviation of signal1
    dem2 = torch.sqrt(torch.sum((signal2 - mean_signal2)**2))  # Standard deviation of signal2
    
    # Calculate Pearson correlation coefficient
    val = num / (dem1 * dem2)
    
    # Return the Pearson correlation coefficient
    return val

# Function to calculate the complete Pearson loss over a batch of signals
def Complete_Pearson(SIGNAL1, SIGNAL2):
    # Initialize the loss variable
    loss = 0
    
    # Assert that both input signals have the same batch size (first dimension)
    assert SIGNAL1.shape[0] == SIGNAL2.shape[0], 'Error: Signals must have the same batch size.'
    
    # Iterate over each signal in the batch
    for i in range(SIGNAL1.shape[0]):
        # Get the individual signals
        signal1 = SIGNAL1[i]
        signal2 = SIGNAL2[i]
        
        # Add the Pearson loss between the current pair of signals
        loss += Pearson_loss(signal1, signal2)
    
    # Return the total Pearson loss over the entire batch
    return loss


In [ ]:
class Transformer_model(pl.LightningModule):
    # Initialization method for the Transformer model
    def __init__(self):
        super().__init__()
        
        # Instantiate the Transformer_ppg model (you need to define this model elsewhere)
        self.model = Transformer_ppg()
        
        # Mean Squared Error loss (MSE) for training
        self.mse = nn.MSELoss()
        
        # Adam optimizer with a learning rate of 1e-4
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        
    # Forward pass: Given the input 'face', pass it through the model
    def forward(self, face):
        ypred = self.model(face)  # Output prediction from the model
        return ypred
    
    # Training step: Called during each batch of training
    def training_step(self, batch, batch_index):
        face, ppg = batch  # Unpack the batch into face data and PPG (Photoplethysmogram) signal
        ypred = self(face)  # Forward pass through the model
        
        # Compute loss: MSE loss minus the Pearson loss (to encourage better correlation)
        loss = self.mse(ypred, ppg[:, 0, :]) - Complete_Pearson(ypred, ppg[:, 0, :])
        
        # Log the training loss for visualization
        self.log('Training_loss', loss)
        
        return loss  # Return the computed loss
    
    # Validation step: Called during each batch of validation
    def validation_step(self, batch, batch_index):
        face, ppg = batch  # Unpack the batch into face data and PPG (Photoplethysmogram) signal
        ypred = self(face)  # Forward pass through the model
        
        # Compute loss: MSE loss minus the Pearson loss
        loss = self.mse(ypred, ppg[:, 0, :]) - Complete_Pearson(ypred, ppg[:, 0, :])
        
        # Prepare for calculating MAE by comparing predicted HR (Heart Rate) and true HR
        ytest = []  # List to store predicted HR values
        ground = []  # List to store ground truth HR values
        
        # Loop through the predicted and actual PPG values
        for pred, LABEL in zip(ypred, ppg[:, 0, :]):
            pred = torch.squeeze(pred)  # Remove unnecessary dimensions
            pred = pred.cpu().detach().numpy()  # Convert tensor to NumPy array
            
            # Filter the predicted signal to remove noise
            pred = filtering(data=pred, fs=30, lowerCutoff=.7, higherCutoff=3, filterOrder=5)
            
            # Compute HR from the filtered signal
            hr2 = HR_FINDER(pred, 30)
            
            # Detrend the ground truth PPG signal
            LABEL = scipy.signal.detrend(LABEL.cpu().detach().numpy())
            
            # Compute HR from the ground truth signal
            labelhr = HR_FINDER(LABEL, 30)
            
            # Store the HR values
            ground.append(labelhr)
            ytest.append(hr2)
        
        # Calculate Mean Absolute Error (MAE) between predicted HR and ground truth HR
        MAE = 0
        for i in range(len(ground)):
            MAE += abs(ytest[i] - ground[i])  # Compute absolute error
        MAE = MAE / i  # Average the errors
        
        # Log the validation MAE loss
        self.log('Val loss', MAE)
        
        return loss  # Return the computed loss
    
    # Optimizer configuration: Return the optimizer used for training
    def configure_optimizers(self):
        return [self.optimizer]  # Return the optimizer


In [ ]:
# Create a ModelCheckpoint callback for PyTorch Lightning
checkpoint = ModelCheckpoint(
    # The metric to monitor during training (in this case, validation loss)
    monitor='Val loss', 
    
    # Save the top 'k' best models based on the monitored metric (only the best one is saved here)
    save_top_k=1,  
    
    # Mode indicates whether we want to minimize ('min') or maximize ('max') the monitored metric
    mode='min'  
)


In [ ]:
model=Transformer_model() ### model is intialised

In [ ]:
# Create a PyTorch Lightning Trainer
trainer = pl.Trainer(
    # Automatically choose the appropriate hardware (CPU or GPU) for training
    accelerator='auto',  
    
    # Set up a TensorBoard logger to log training progress (will save logs in the './logs' directory)
    logger=TensorBoardLogger(save_dir='.\logs'),  
    
    # Set the logging interval (log every 10 steps)
    log_every_n_steps=10,  
    
    # Include the ModelCheckpoint callback to monitor the validation loss and save the best model
    callbacks=checkpoint,  
    
    # Set the number of training epochs
    max_epochs=100  
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# Start the training process using the trainer
trainer.fit(model, train_loader, val_loader)


You are using a CUDA device ('NVIDIA GeForce RTX 3080 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | Transformer_ppg | 4.4 M 
1 | mse   | MSELoss         | 0     
------------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
17.486    Total estimated model params size (MB)


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\PhysioSens\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
C:\Users\PhysioSens\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=1000` reached.


# Loading the best weight for 1D-CXNet

In [ ]:
# Load the trained model from the checkpoint
model = Transformer_model.load_from_checkpoint(
    r'C:\Users\PhysioSens\Desktop\python-jupyter\transformer_ppg\ubfc\3D-CNN\logs\lightning_logs\version_18\checkpoints\epoch=42-step=1548.ckpt',
    map_location=torch.device('cuda')  # Load the model on GPU
)

# Switch the model to evaluation mode, which disables dropout layers and batch norm updates
model = model.eval()


In [ ]:
import time  # Import the time module to track processing time

ytest = []  # List to store predicted heart rates
ground = []  # List to store ground truth heart rates
time_req = []  # List to store the time taken for each prediction

# Loop through each batch in the validation data
for face, ppg in val_data:
    face = face.unsqueeze(0).to('cuda')  # Add a batch dimension and move to GPU

    # Perform inference with no gradient calculation (no backpropagation)
    with torch.no_grad():
        tic = time.time()  # Record the start time
        pred = model(face)  # Get the model's prediction for the input face data
        toc = time.time()  # Record the end time
        time_req.append(toc - tic)  # Calculate and store the time taken for prediction

    pred = torch.squeeze(pred)  # Remove unnecessary dimensions from the predicted tensor
    pred = pred.cpu().detach().numpy()  # Move the tensor to CPU and convert it to a NumPy array

    # Apply filtering to the predicted signal (bandpass filter between 0.7 Hz and 3 Hz)
    pred = filtering(data=pred, fs=30, lowerCutoff=.7, higherCutoff=3, filterOrder=5)

    # Process the filtered signal using the HeartPy library (assuming it's imported correctly)
    working_data, measures = hp.process(pred, 30)  # Process the signal to extract heart rate info
    hr1 = measures['bpm']  # Get the heart rate from HeartPy
    hr2 = HR_FINDER(pred, 30)  # Calculate heart rate using the HR_FINDER function

    # Store both heart rate values in a list
    HR = [hr1, hr2]

    # Detrend the ground truth signal (removes any linear trend)
    label = scipy.signal.detrend(ppg[0, :].detach().numpy())
    labelhr = HR_FINDER(label, 30)  # Calculate heart rate from the ground truth

    # Append the ground truth heart rate to the ground list
    ground.append(labelhr)

    # Append the predicted heart rate to the ytest list
    ytest.append(hr2)  # Ensure `hr2` is the final prediction being used



C:\ProgramData\Anaconda3\lib\site-packages\numpy\ma\core.py:5334: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\PhysioSens\AppData\Roaming\Python\Python39\site-packages\heartpy\analysis.py:677: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  interp = UnivariateSpline(x, rrlist, k=3)


In [14]:
predict_hr_fft_all = np.array(ytest)
gt_hr_fft_all = np.array(ground)
num_test_samples = len(predict_hr_fft_all)
METRIC=[]
for metric in ['MAE','RMSE','MAPE','Pearson', 'SNR']:
    if metric == "MAE":
        MAE_FFT = np.mean(np.abs(predict_hr_fft_all - gt_hr_fft_all))
        standard_error = np.std(np.abs(predict_hr_fft_all - gt_hr_fft_all)) / np.sqrt(num_test_samples)
        METRIC.append(MAE_FFT)
        print("FFT MAE (FFT Label): {0} +/- {1}".format(MAE_FFT, standard_error))
    elif metric == "RMSE":
        RMSE_FFT = np.sqrt(np.mean(np.square(predict_hr_fft_all - gt_hr_fft_all)))
        standard_error = np.std(np.square(predict_hr_fft_all - gt_hr_fft_all)) / np.sqrt(num_test_samples)
        print("FFT RMSE (FFT Label): {0} +/- {1}".format(RMSE_FFT, standard_error))
        METRIC.append(RMSE_FFT)
    elif metric == "MAPE":
        MAPE_FFT = np.mean(np.abs((predict_hr_fft_all - gt_hr_fft_all) / gt_hr_fft_all)) * 100
        standard_error = np.std(np.abs((predict_hr_fft_all - gt_hr_fft_all) / gt_hr_fft_all)) / np.sqrt(num_test_samples) * 100
        print("FFT MAPE (FFT Label): {0} +/- {1}".format(MAPE_FFT, standard_error))
        METRIC.append(MAPE_FFT)
    elif metric == "Pearson":
        Pearson_FFT = np.corrcoef(predict_hr_fft_all, gt_hr_fft_all)
        correlation_coefficient = Pearson_FFT[0][1]
        standard_error = np.sqrt((1 - correlation_coefficient**2) / (num_test_samples - 2))
        METRIC.append(Pearson_FFT)
        print("FFT Pearson (FFT Label): {0} +/- {1}".format(correlation_coefficient, standard_error))

FFT MAE (FFT Label): 1.5005189567065218 +/- 0.3307579391099182
FFT RMSE (FFT Label): 4.294517161859201 +/- 11.883669575200638
FFT MAPE (FFT Label): 1.6394707500858106 +/- 0.4157076190376846
FFT Pearson (FFT Label): 0.967786306744515 +/- 0.020836885182161924
